In [1]:
import base64
from datetime import datetime
from urllib import request
import gzip 
import pandas as pd
import os
import json
import math
import os
import glob
import tempfile

In [13]:
os.chdir('S:/Projects/Intern Projects/Colorado Intern/3ce/GZIP FILES/2008')

In [14]:
def checkForNone(reading):
    if reading in ("-9999", "-99999", "-999999", ""):
        return None
    else:
        return False

def convertMeasurementNoScaling(reading):
    if checkForNone(reading) is None:
        return None
    else:
        return float(reading)

# convert from deca to standard
def convertMeasurement(reading):
    if checkForNone(reading) is None:
        return None
    else:
        r = float(reading)
        return r / 10

def cloudCoverLookup(reading):
    return {
        "0": "None, SKC or CLR",
        "1": "One okta - 1/10 or less but not zero",
        "2": "Two oktas - 2/10 - 3/10, or FEW",
        "3": "Three oktas - 4/10",
        "4": "Four oktas - 5/10, or SCT",
        "5": "Five oktas - 6/10",
        "6": "Six oktas - 7/10 - 8/10",
        "7": "Seven oktas - 9/10 or more but not 10/10, or BKN",
        "8": "Eight oktas - 10/10, or OVC",
        "9": "Sky obscured, or cloud amount cannot be estimated",
        "10": "Partial obscuration",
        "11": "Thin scattered",
        "12": "Scattered",
        "13": "Dark scattered",
        "14": "Thin broken",
        "15": "Broken",
        "16": "Dark broken",
        "17": "Thin overcast",
        "18": "Overcast",
        "19": "Dark overcast",
    }.get(reading)  
    
# Defining M/D/Y
year = 2008
month = 1
day = datetime.today().day

# convert months and days to strings in a pretty way
if month < 10:
    month = '0'+str(month)
else:
    month = str(month)

if day < 10:
    day = '0'+str(day)
else:
    day = str(day)

stryear = str(year)

dateid = stryear+month+day

locations = {'Santa Ynez': '723762-93244', 'Fresno County':'723890-93193', 'San Luis':'722897-93206', 'Santa Cruz': '745058-23277', 'San Benito': '724917-23233'}

# column names for the resulting data frame
col_names = ["StationID","DateTime","airTemp","dewPoint","pressure","windDirection","windSpeed","cloudCover","oneHourPrecipitation"]

# ftp path to NOAA data
ftppath = 'https://www.ncei.noaa.gov/pub/data/noaa/isd-lite/'+stryear+'/'

# loop through locations
for ii in locations:

    file_out = 'Weather_'+locations[ii]+'_'+stryear+'-'+dateid+'.csv'

    # make full URL to file on FTP site
    url = ftppath+locations[ii]+'-'+stryear+'.gz'
    print('Working on '+ii+' - '+url)

    # fetch the FTP file - comes down as a gz file
    ftpfile = request.urlopen(url)

    # read the GZ file and pull out the readings
    with gzip.open(ftpfile) as txt:
        rowlist = []
        count = 0
        for line in txt:
            l = line.split()
    #         print(line)
            #convert to correct types etc
            try:
                year          = int(l[0])                                  # l[0] # year
            except:
    #                 print('Skippped line '+str(count))
    #                 print(l)

                continue
            count += 1
            month         = int(l[1])                                  # l[1] # month
            day           = int(l[2])                                  # l[2] # day
            hour          = int(l[3])                                  # l[3] # hour

            airTemp       = convertMeasurement(l[4])             # l[4] # air temp degrees C * 10 or -9999
            dewPt         = convertMeasurement(l[5])             # l[5] # dew point degrees C * 10 or -9999
            pressure      = convertMeasurement(l[6])             # l[6] # sea level pressure hectopascals * 10 or -9999
            windDirection = convertMeasurementNoScaling(l[7])    # l[7] # wind direction angular degrees
            windSpeed     = convertMeasurement(l[8])             # l[8] # wind speed rate meters per second * 10
            cloudCover    = cloudCoverLookup(l[9])               # l[9] # MISSING VALUE: -9999
                                                                    # DOMAIN:
                                                                    # 0: None, SKC or CLR
                                                                    # 1: One okta - 1/10 or less but not zero
                                                                    # 2: Two oktas - 2/10 - 3/10, or FEW
                                                                    # 3: Three oktas - 4/10
                                                                    # 4: Four oktas - 5/10, or SCT
                                                                    # 5: Five oktas - 6/10
                                                                    # 6: Six oktas - 7/10 - 8/10
                                                                    # 7: Seven oktas - 9/10 or more but not 10/10, or BKN
                                                                    # 8: Eight oktas - 10/10, or OVC
                                                                    # 9: Sky obscured, or cloud amount cannot be estimated
                                                                    # 10: Partial obscuration
                                                                    # 11: Thin scattered
                                                                    # 12: Scattered
                                                                    # 13: Dark scattered
                                                                    # 14: Thin broken
                                                                    # 15: Broken
                                                                    # 16: Dark broken
                                                                    # 17: Thin overcast
                                                                    # 18: Overcast
                                                                    # 19: Dark overcast
            precipShortDuration = convertMeasurement(l[10])      # l[10] # The depth of liquid precipitation that is measured over a one hour accumulation period. UNITS: cm
            precipLongDuration  = convertMeasurement(l[11])      # l[11] # The depth of liquid precipitation that is measured over a six hour accumulation period. UNITS: cm


            dt = datetime(year, month, day, hour)
    #         result.iloc[i]['timestamp'] = dt
            reading = {
                    "StationID": locations[ii],
                    "DateTime": dt,
                    "airTemp": airTemp,
                    "dewPoint": dewPt,
                    "pressure": pressure,
                    "windDirection": windDirection,
                    "windSpeed": windSpeed,
                    "cloudCover": cloudCover,
                    "oneHourPrecipitation": precipShortDuration,
                    "sixHourPrecipitation": precipLongDuration
                }
            rowlist.append(reading)
            
                       # make a data frame of the individual rows
        df = pd.DataFrame(rowlist,columns=col_names)
        df1 = df.to_csv(file_out)


Working on Santa Ynez - https://www.ncei.noaa.gov/pub/data/noaa/isd-lite/2008/723762-93244-2008.gz
Working on Fresno County - https://www.ncei.noaa.gov/pub/data/noaa/isd-lite/2008/723890-93193-2008.gz
Working on San Luis - https://www.ncei.noaa.gov/pub/data/noaa/isd-lite/2008/722897-93206-2008.gz
Working on Santa Cruz - https://www.ncei.noaa.gov/pub/data/noaa/isd-lite/2008/745058-23277-2008.gz
Working on San Benito - https://www.ncei.noaa.gov/pub/data/noaa/isd-lite/2008/724917-23233-2008.gz


In [15]:
df4 = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "*.csv"))))
df4.head()

,Unnamed: 0,StationID,DateTime,airTemp,dewPoint,pressure,windDirection,windSpeed,cloudCover,oneHourPrecipitation
0,0,722897-93206,2008-01-01 00:00:00,18.0,1.0,1019.6,350.0,3.6,NaN,-999.9
1,1,722897-93206,2008-01-01 01:00:00,14.4,1.1,1020.0,0.0,0.0,NaN,0.0
2,2,722897-93206,2008-01-01 02:00:00,11.1,1.7,1020.6,270.0,1.5,NaN,0.0
3,3,722897-93206,2008-01-01 03:00:00,10.0,1.7,1021.1,0.0,0.0,NaN,0.0
4,4,722897-93206,2008-01-01 04:00:00,8.3,1.1,1021.7,0.0,0.0,NaN,0.0


In [16]:
df4.drop(columns = 'Unnamed: 0', axis = 1, inplace= True)
df4.head()

,StationID,DateTime,airTemp,dewPoint,pressure,windDirection,windSpeed,cloudCover,oneHourPrecipitation
0,722897-93206,2008-01-01 00:00:00,18.0,1.0,1019.6,350.0,3.6,NaN,-999.9
1,722897-93206,2008-01-01 01:00:00,14.4,1.1,1020.0,0.0,0.0,NaN,0.0
2,722897-93206,2008-01-01 02:00:00,11.1,1.7,1020.6,270.0,1.5,NaN,0.0
3,722897-93206,2008-01-01 03:00:00,10.0,1.7,1021.1,0.0,0.0,NaN,0.0
4,722897-93206,2008-01-01 04:00:00,8.3,1.1,1021.7,0.0,0.0,NaN,0.0


In [17]:
df4.loc[:,'Year'] = df['DateTime'].dt.year
df4.head()


,StationID,DateTime,airTemp,dewPoint,pressure,windDirection,windSpeed,cloudCover,oneHourPrecipitation,Year
0,722897-93206,2008-01-01 00:00:00,18.0,1.0,1019.6,350.0,3.6,NaN,-999.9,2008.0
1,722897-93206,2008-01-01 01:00:00,14.4,1.1,1020.0,0.0,0.0,NaN,0.0,2008.0
2,722897-93206,2008-01-01 02:00:00,11.1,1.7,1020.6,270.0,1.5,NaN,0.0,2008.0
3,722897-93206,2008-01-01 03:00:00,10.0,1.7,1021.1,0.0,0.0,NaN,0.0,2008.0
4,722897-93206,2008-01-01 04:00:00,8.3,1.1,1021.7,0.0,0.0,NaN,0.0,2008.0


In [18]:
df4['Year'] = df4['Year'].astype(str)
df4.dtypes
df4['Year'] = df4['Year'].replace('2008.0','2008')
df4.head()

,StationID,DateTime,airTemp,dewPoint,pressure,windDirection,windSpeed,cloudCover,oneHourPrecipitation,Year
0,722897-93206,2008-01-01 00:00:00,18.0,1.0,1019.6,350.0,3.6,NaN,-999.9,2008
1,722897-93206,2008-01-01 01:00:00,14.4,1.1,1020.0,0.0,0.0,NaN,0.0,2008
2,722897-93206,2008-01-01 02:00:00,11.1,1.7,1020.6,270.0,1.5,NaN,0.0,2008
3,722897-93206,2008-01-01 03:00:00,10.0,1.7,1021.1,0.0,0.0,NaN,0.0,2008
4,722897-93206,2008-01-01 04:00:00,8.3,1.1,1021.7,0.0,0.0,NaN,0.0,2008


In [19]:
df4.loc[:,'Month'] = df['DateTime'].dt.month
df4['Month'] = df4['Month'].astype(str)
df4.dtypes
df4['Month'] = df4['Month'].replace(['1.0','2.0','3.0','4.0','5.0','6.0','7.0','8.0','9.0','10.0','11.0','12.0'], ['1','2','3','4','5','6','7','8','9','10','11','12'])
df4.head()

,StationID,DateTime,airTemp,dewPoint,pressure,windDirection,windSpeed,cloudCover,oneHourPrecipitation,Year,Month
0,722897-93206,2008-01-01 00:00:00,18.0,1.0,1019.6,350.0,3.6,NaN,-999.9,2008,1
1,722897-93206,2008-01-01 01:00:00,14.4,1.1,1020.0,0.0,0.0,NaN,0.0,2008,1
2,722897-93206,2008-01-01 02:00:00,11.1,1.7,1020.6,270.0,1.5,NaN,0.0,2008,1
3,722897-93206,2008-01-01 03:00:00,10.0,1.7,1021.1,0.0,0.0,NaN,0.0,2008,1
4,722897-93206,2008-01-01 04:00:00,8.3,1.1,1021.7,0.0,0.0,NaN,0.0,2008,1


In [20]:
df4.loc[:,'Day'] = df['DateTime'].dt.day
df4['Day'] = df4['Day'].astype(str)
df4.dtypes
df4['Day'] = df4['Day'].replace(['1.0','2.0','3.0','4.0','5.0','6.0','7.0','8.0','9.0','10.0','11.0','12.0','13.0','14.0','15.0','16.0','17.0','18.0','19.0','20.0','21.0','22.0','23.0','24.0','25.0','26.0','27.0','28.0','29.0','30','31.0'],['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31'])
df4.head()

,StationID,DateTime,airTemp,dewPoint,pressure,windDirection,windSpeed,cloudCover,oneHourPrecipitation,Year,Month,Day
0,722897-93206,2008-01-01 00:00:00,18.0,1.0,1019.6,350.0,3.6,NaN,-999.9,2008,1,1
1,722897-93206,2008-01-01 01:00:00,14.4,1.1,1020.0,0.0,0.0,NaN,0.0,2008,1,1
2,722897-93206,2008-01-01 02:00:00,11.1,1.7,1020.6,270.0,1.5,NaN,0.0,2008,1,1
3,722897-93206,2008-01-01 03:00:00,10.0,1.7,1021.1,0.0,0.0,NaN,0.0,2008,1,1
4,722897-93206,2008-01-01 04:00:00,8.3,1.1,1021.7,0.0,0.0,NaN,0.0,2008,1,1


In [21]:
df4.loc[:,'Hour'] = df['DateTime'].dt.hour
df4['Hour'] = df4['Hour'].astype(str)
df4.dtypes
df4['Hour'] = df4['Hour'].replace(['1.0','2.0','3.0','4.0','5.0','6.0','7.0','8.0','9.0','10.0','11.0','12.0','13.0','14.0','15.0','16.0','17.0','18.0','19.0','20.0','21.0','22.0','23.0','24.0'],['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24'])
df4.head()

,StationID,DateTime,airTemp,dewPoint,pressure,windDirection,windSpeed,cloudCover,oneHourPrecipitation,Year,Month,Day,Hour
0,722897-93206,2008-01-01 00:00:00,18.0,1.0,1019.6,350.0,3.6,NaN,-999.9,2008,1,1,1
1,722897-93206,2008-01-01 01:00:00,14.4,1.1,1020.0,0.0,0.0,NaN,0.0,2008,1,1,2
2,722897-93206,2008-01-01 02:00:00,11.1,1.7,1020.6,270.0,1.5,NaN,0.0,2008,1,1,3
3,722897-93206,2008-01-01 03:00:00,10.0,1.7,1021.1,0.0,0.0,NaN,0.0,2008,1,1,4
4,722897-93206,2008-01-01 04:00:00,8.3,1.1,1021.7,0.0,0.0,NaN,0.0,2008,1,1,5


In [22]:
df5=df4.reindex(columns=['Year','Month','Day','Hour','airTemp','dewPoint','pressure','windDirection','windSpeed','cloudCover','oneHourPrecipitation','sixHourPercipitation','StationID'])
df5.head()

,Year,Month,Day,Hour,airTemp,dewPoint,pressure,windDirection,windSpeed,cloudCover,oneHourPrecipitation,sixHourPercipitation,StationID
0,2008,1,1,1,18.0,1.0,1019.6,350.0,3.6,NaN,-999.9,NaN,722897-93206
1,2008,1,1,2,14.4,1.1,1020.0,0.0,0.0,NaN,0.0,NaN,722897-93206
2,2008,1,1,3,11.1,1.7,1020.6,270.0,1.5,NaN,0.0,NaN,722897-93206
3,2008,1,1,4,10.0,1.7,1021.1,0.0,0.0,NaN,0.0,NaN,722897-93206
4,2008,1,1,5,8.3,1.1,1021.7,0.0,0.0,NaN,0.0,NaN,722897-93206


In [23]:
values = ['nan', ' ', -999]
df6= df5[df5.Year.isin(values) == False]
df7 = df6[df6.Month.isin(values) == False]
df8 = df7[df7.Day.isin(values) == False]
df9 = df8[df8.Hour.isin(values) == False]
df10 = df9[df9.airTemp.isin(values) == False]

In [24]:
df10.to_csv('all_data2008.csv', header = None, index = False)